In [61]:
class RegresionLineal:  
    
    def __init__(self, path, y, **kwargs):
        self.path = path
        self.y = y
        self.kwargs = kwargs
        print('Se creo la regresión lineal')
        
    def predictors_vars(self):
        # Esta función almacena y coloca en una lista todas las variables predictoras
        l = []
        for k, v in self.kwargs.items():
            l.append(v)
        return l
        
    def join_predictors_vars(self):
        # Esta función junta las variables predictoras para agregarlas en el modelo_multiple
        var_x = self.predictors_vars()
        modelo_variables = '+'.join(var_x)
        return modelo_variables
    
    def model(self):
        
        import pandas as pd
        import statsmodels.formula.api as smf
        
        data = pd.read_csv(self.path)     
        variables_x = self.y + '~' + self.join_predictors_vars()
        lm = smf.ols(formula = variables_x, data = data).fit()
        return lm             
        
    def parameters_pvalues_rsquared(self):
        model = self.model()
        print('Parametros del modelo:\n{}\n-----------'.format(model.params))
        print('P_valores del modelo:\n{}\n-----------'.format(model.pvalues))
        print('R_cuadrada del modelo:\n{}\n-----------'.format(model.rsquared))
        print('R_cuadrada ajustada del modelo:\n{}\n-----------'.format(model.rsquared_adj))
        #print('Según el modelo: Sales = {} + {}*"TV" + {}*"Newspaper" '.format(model.params[0], model.params[1], model.params[2]))
       
    def prediccion_y(self):
        
        import pandas as pd
        
        """
         Llamamos a la función 'variables_x()' para obtener los nombre de
         las variables x y pasarlas por la función 'predict'.         
        """
        model = self.model()
        data = pd.read_csv(self.path)
        y_pred = model.predict(data[self.predictors_vars()])
        return y_pred
    
    def error(self):
        
        import pandas as pd
        import numpy as np
        
        count_var   = len(self.predictors_vars())
        data        = pd.read_csv(self.path) 
        SSD         = sum((data[self.y]-self.prediccion_y())**2)
        RSE         = np.sqrt(SSD/(len(data)-count_var-1))
        e           = RSE / np.mean(data[self.y])
        # La salida de esta función, detalla el porcentaje de error que no puede explicar el modelo, entre menor sea es mejor para el modelo.
        return e
    
    def multicolinealidad(self, com=None, **kwargs):
        
        l = []
        for i in kwargs:
            l.append(i)
          
        j = '+'.join(l)
        
        var = com + '~' + j
        
        import pandas as pd
        import statsmodels.formula.api as smf
        
        data = pd.read_csv(self.path)
        f = smf.ols(formula=var, data = data).fit()
        VIF = 1/(1-f.rsquared)
        
        if VIF == 1:
            print('Las variables no están correlacionadas')
        elif VIF < 5:
            print('Las variables tienen una correlación moderada y se pueden quedar en el modelo')
        elif VIF > 5:
            print('Las variables están altamente correlacionadas y deben desaparecer del modelo')
        return 'El factor inflación de la varianza es de: {}'.format(VIF)
    
    # Regresión lineal con la librería de sklearn
    
    def seleccion_variables(self):
        
        import pandas as pd
        
        data = pd.read_csv(self.path)  
        
        X = data[self.predictors_vars()]
        Y = data[self.y]
        
        from sklearn.feature_selection import RFE
        from sklearn.svm import SVR
        
        import numpy as np
        
        estimator = SVR(kernel="linear")
        # El numero dos es un indice que elige la cantidad de variables que debería tomar el modelo
        selector = RFE(estimator, 2, step=1)
        selector = selector.fit(X,Y)
        
        return self.predictors_vars(), selector.support_, selector.ranking_ 